In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression

    


In [8]:
#1. data loading
df = pd.read_csv("admission.csv")
#df.head()

#상관계수 이용 확인>><<?????????????????????
# 랭크 조절 해 (음의 상관계수)
# 랭크 값 역순으로 조절
df.corr()

def change_rank(k):
    if k == 4:
        return 1
    elif k == 3:
        return 2
    elif k == 2:
        return 3
    elif k == 1:
        return 4

###################### 이상치 처리 작업
def process_outlier(tmp_df, tmp):
    q1, q3 = np.percentile(tmp,[25,75])
    iqr = q3 - q1
    upper = q3 + iqr * 1.5
    lower = q1 - iqr * 1.5
    upper_mask = tmp > upper
    lower_mask = tmp < lower
    
    result_mask = upper_mask | lower_mask 
    tmp_df = tmp_df.loc[~result_mask]
    
    return tmp_df

# 랭크 높으면 높을 수록 합격이 되어야
df ['rank'] = df['rank'].apply(lambda x : change_rank(x))

# 결측치 처리 (결측)
# 이상치 처리

df = process_outlier(df, df["gre"])
df = process_outlier(df, df["gpa"])

df.shape

###########################################

#df.isnull().sum()
# 머신 러닝
x_data = df[["gre", "gpa", "rank"]].values # numpy arr
x_data = MinMaxScaler().fit_transform(x_data)

y_data = df["admit"].values.reshape(-1,1)

# placeholder
X = tf.placeholder(shape=[None, 3], dtype=tf.float32)
Y = tf.placeholder(shape=[None, 1], dtype=tf.float32)

# Weight & bias
W = tf.Variable(tf.random_normal([3,1]), name="weight")
b = tf.Variable(tf.random_normal([1]), name="bias")

# Hypothesis
logit = tf.matmul(X,W) + b
H = tf.sigmoid(logit)

# 
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logit, labels= Y))

#train
# train node
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.01)
train = optimizer.minimize(cost)

# session & initialize
sess = tf.Session()
sess.run(tf.global_variables_initializer())

## 학습
for step in range(30000):
    _, cost_val = sess.run([train,cost], feed_dict={ X : x_data,
                                                     Y : y_data})
    if step % 3000 == 0:
        print("cost val: {}".format(cost_val))
# 학습이 종료
# Linear Regress를 할 떄는 내가 낸 답이 정확한지 
##

## 정확도를 측정해보아요
## 테스트 데이타 셋 없음
## train data set을 test data set 사용

# predict 결과를 알아봐요
# 특정한 X값을 -> training data set사용
# True or False 로 나오는데 1,0 으로 만들어진 예측값
predict = tf.cast(H > 0.5, dtype=tf.float32)
correct = tf.equal(predict, Y) # true, false, true, true

accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32))
print("정확도: {}".format(sess.run(accuracy, feed_dict= { X: x_data,
                                                     Y : y_data})))

# 그 결과 정확도 70% 정도인 모델이 만들어졋따는 걸 알수있따
# 그런데 정확도가 70%면 사용할 수 없다. 95 %가 정말 좋고 아니면 최소는 85 프로 정도이다 
# 정확도 30%

# 프리딕트



cost val: 0.8639286160469055
cost val: 0.6189047694206238
cost val: 0.6018378734588623
cost val: 0.5918797254562378
cost val: 0.5859391093254089
cost val: 0.5823188424110413
cost val: 0.5800711512565613
cost val: 0.5786541700363159
cost val: 0.5777497291564941
cost val: 0.5771664381027222
정확도: 0.701265811920166
